In [4]:
import single_head
import multi_head
import copy
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.ndimage.morphology import binary_fill_holes
import sys
import pickle
np.set_printoptions(threshold=sys.maxsize)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
# load heads
all_heads=[single_head.SingleHead.load_from_pickle(2,i) for i in range (1,16)]

In [ ]:
# make head from first to last
def make_mhead(all_heads, first, last):
    all_heads[first].reset_positions()
    all_heads[first].reset_colors()
    all_heads[first+1].reset_positions()
    all_heads[first+1].reset_colors()   
    heads = multi_head.MultiHead.joined_heads(all_heads[first], all_heads[first+1])
    heads.links.append(multi_head.Link(left=all_heads[first+1].frame_id,right=all_heads[first].frame_id))
    for i in range(first+2,last+1):
        heads.links.append(multi_head.Link(left=all_heads[i].frame_id,right=all_heads[i-1].frame_id))
        all_heads[i].reset_positions()
        all_heads[i].reset_colors()
        heads.append(all_heads[i])        
        if i==last:
             heads.links.append(multi_head.Link(left=all_heads[first].frame_id,right=all_heads[i].frame_id))
    print ("completed")
    return heads

In [34]:
mhead = make_mhead(all_heads, 0,14)
mhead.calc_keypoints()

completed


In [5]:
mhead = multi_head.MultiHead.load_from_pickle(1)

In [8]:
# find the best matches
def calc_connections(mhead):
    for link in mhead.links:
        link.reset()
        mhead.ransac_from_link(link)        

In [ ]:
mhead.save()

In [10]:
for i in range(1,5):
    print("head{}".format(i))
    mhead = multi_head.MultiHead.load_from_pickle(i)
    calc_connections(mhead)
    mhead.save()

head1


/Users/caseliang/Repositories/AV_CW/SIFT.py:91: RuntimeWarning: invalid value encountered in sqrt
  err = np.sqrt(np.var(dist) / (np.sum(inliers) - min_num_inliers))
/Users/caseliang/Repositories/AV_CW/SIFT.py:91: RuntimeWarning: divide by zero encountered in double_scalars
  err = np.sqrt(np.var(dist) / (np.sum(inliers) - min_num_inliers))



Saving Completed
head2



Saving Completed
head3



Saving Completed
head4



Saving Completed


In [18]:
mhead = multi_head.MultiHead.load_from_pickle(4)

In [14]:
# transform from links
only_first_n=15
mhead.reset_all_head_positions()
link_index,err=mhead.get_next_unpositioned_link()
positioned_head_count=0
joined_heads=set() 
while(not link_index is None) and (positioned_head_count<only_first_n or only_first_n==-1):
    mhead.links[link_index].print_short()
    joined_heads.add(mhead.links[link_index].left)
    joined_heads.add(mhead.links[link_index].right)
    foo= mhead.transform_from_link(mhead.links[link_index])
    link_index,err=mhead.get_next_unpositioned_link()
    positioned_head_count=max(positioned_head_count+1,2)
joined_heads=sorted(joined_heads)
print(joined_heads)
no_heads= len(joined_heads)
joined_heads=[str(h) for h in joined_heads]
name= f"Seq_{mhead.heads[0].sequence_id}_frames_{'_'.join(joined_heads)}"
mhead.create_spheres(min(2/max(no_heads,0.2),1),name=name)

any_head_positioned False
2-1, Count=155, Err=0.0007


[1, 0, 0], -0.0, 7056 ,0|| 7056, 0 , 0
[1, 0, 0], -0.0, 7056 ,0|| 7056, 7056 , 0
[1, 0, 0], -0.0, 7056 ,0|| 7056, 7056 , 7056
[1, 0, 0], -0.0, 7057 ,0|| 7057, 7056 , 7056
[1, 0, 0], -0.1, 7057 ,0|| 7057, 7057 , 7056
[1, 0, 0], -0.1, 7057 ,0|| 7057, 7057 , 7057
[1, 0, 0], -0.1, 7058 ,0|| 7058, 7057 , 7057
[1, 0, 0], -0.1, 7059 ,0|| 7059, 7058 , 7057
[1, 0, 0], -0.1, 7060 ,0|| 7060, 7059 , 7058
[1, 0, 0], -0.1, 7060 ,0|| 7060, 7060 , 7059
[1, 0, 0], -0.1, 7060 ,0|| 7060, 7060 , 7060
[1, 0, 0], -0.1, 7060 ,0|| 7060, 7060 , 7060
[1, 0, 0], -0.1, 7060 ,0|| 7060, 7060 , 7060
[1, 0, 0], -0.1, 7060 ,0|| 7060, 7060 , 7060
[1, 0, 0], -0.2, 7060 ,0|| 7060, 7060 , 7060
[1, 0, 0], -0.2, 7061 ,0|| 7061, 7060 , 7060
[1, 0, 0], -0.2, 7061 ,0|| 7061, 7061 , 7060
[1, 0, 0], -0.2, 7061 ,0|| 7061, 7061 , 7061
[1, 0, 0], -0.2, 7062 ,0|| 7062, 7061 , 7061
[1, 0, 0], -0.2, 7062 ,0|| 7062, 7062 , 7061
[1, 0, 0], -0.2, 7062 ,0|| 7062, 7062 , 7062
[1, 0, 0], -0.2, 7062 ,0|| 7062, 7062 , 7062
[1, 0, 0], -0.2, 70

[1, 0, 0], -0.0, 7085 ,0|| 7085, 0 , 0
[1, 0, 0], -0.0, 7085 ,0|| 7085, 7085 , 0
[1, 0, 0], -0.0, 7085 ,0|| 7085, 7085 , 7085
[1, 0, 0], -0.0, 7086 ,0|| 7086, 7085 , 7085
[1, 0, 0], -0.1, 7085 ,0|| 7085, 7086 , 7085
[0, 1, 0], -0.0, 6943 ,0|| 6943, 0 , 0
[0, 1, 0], -0.0, 6945 ,0|| 6945, 6943 , 0
[0, 1, 0], -0.0, 6946 ,0|| 6946, 6945 , 6943
[0, 1, 0], -0.0, 6946 ,0|| 6946, 6946 , 6945
[0, 1, 0], -0.1, 6947 ,0|| 6947, 6946 , 6946
[0, 1, 0], -0.1, 6947 ,0|| 6947, 6947 , 6946
[0, 1, 0], -0.1, 6947 ,0|| 6947, 6947 , 6947
[0, 1, 0], -0.1, 6947 ,0|| 6947, 6947 , 6947
[0, 1, 0], -0.1, 6948 ,0|| 6948, 6947 , 6947
[0, 1, 0], -0.1, 6949 ,0|| 6949, 6948 , 6947
[0, 1, 0], -0.1, 6951 ,0|| 6951, 6949 , 6948
[0, 1, 0], -0.1, 6952 ,0|| 6952, 6951 , 6949
[0, 1, 0], -0.1, 6952 ,0|| 6952, 6952 , 6951
[0, 1, 0], -0.1, 6952 ,0|| 6952, 6952 , 6952
[0, 1, 0], -0.2, 6952 ,0|| 6952, 6952 , 6952
[0, 1, 0], -0.2, 6952 ,0|| 6952, 6952 , 6952
[0, 1, 0], -0.2, 6953 ,0|| 6953, 6952 , 6952
[0, 1, 0], -0.2, 6954 ,0|| 6

[1, 0, 0], -0.0, 6568 ,0|| 6568, 0 , 0
[1, 0, 0], -0.0, 6568 ,0|| 6568, 6568 , 0
[1, 0, 0], -0.0, 6566 ,0|| 6566, 6568 , 6568
[1, 0, 0], -0.0, 6568 ,0|| 6568, 6568 , 6568
[1, 0, 0], -0.0, 6566 ,1|| 6566, 6568 , 6568
[1, 0, 0], -0.0, 6568 ,1|| 6568, 6568 , 6568
[0, 1, 0], -0.0, 6703 ,0|| 6703, 0 , 0
[0, 1, 0], -0.0, 6703 ,0|| 6703, 6703 , 0
[0, 1, 0], -0.0, 6703 ,0|| 6703, 6703 , 6703
[0, 1, 0], -0.0, 6702 ,0|| 6702, 6703 , 6703
[0, 1, 0], -0.0, 6703 ,0|| 6703, 6703 , 6703
[0, 1, 0], -0.0, 6702 ,1|| 6702, 6703 , 6703
[0, 1, 0], -0.0, 6703 ,1|| 6703, 6703 , 6703
[0, 0, 1], -0.0, 6756 ,0|| 6756, 0 , 0
[0, 0, 1], -0.0, 6756 ,0|| 6756, 6756 , 0
[0, 0, 1], -0.0, 6759 ,0|| 6759, 6756 , 6756
[0, 0, 1], -0.0, 6759 ,0|| 6759, 6759 , 6756
[0, 0, 1], -0.1, 6760 ,0|| 6760, 6759 , 6759
[0, 0, 1], -0.1, 6760 ,0|| 6760, 6760 , 6759
[0, 0, 1], -0.1, 6759 ,0|| 6759, 6760 , 6760
[0, 0, 1], -0.1, 6759 ,0|| 6760, 6760 , 6759
[0, 0, 1], -0.1, 6759 ,1|| 6759, 6760 , 6760
[0, 0, 1], -0.1, 6759 ,1|| 6760, 6760

[1, 0, 0], -0.0, 6577 ,0|| 6577, 0 , 0
[1, 0, 0], -0.0, 6576 ,0|| 6576, 6577 , 0
[1, 0, 0], -0.0, 6576 ,0|| 6576, 6576 , 6577
[1, 0, 0], -0.0, 6580 ,0|| 6576, 6577 , 6580
[1, 0, 0], -0.0, 6577 ,0|| 6577, 6580 , 6577
[0, 1, 0], -0.0, 6617 ,0|| 6617, 0 , 0
[0, 1, 0], -0.0, 6617 ,0|| 6617, 6617 , 0
[0, 1, 0], -0.0, 6618 ,0|| 6618, 6617 , 6617
[0, 1, 0], -0.0, 6618 ,0|| 6618, 6618 , 6617
[0, 1, 0], -0.1, 6618 ,0|| 6618, 6618 , 6618
[0, 1, 0], -0.1, 6618 ,0|| 6618, 6618 , 6618
[0, 1, 0], -0.1, 6617 ,0|| 6617, 6618 , 6618
[0, 1, 0], -0.1, 6618 ,0|| 6618, 6618 , 6618
[0, 1, 0], -0.1, 6617 ,1|| 6617, 6618 , 6618
[0, 1, 0], -0.1, 6618 ,1|| 6618, 6618 , 6618
[0, 0, 1], -0.0, 6628 ,0|| 6628, 0 , 0
[0, 0, 1], -0.0, 6627 ,0|| 6627, 6628 , 0
[0, 0, 1], -0.0, 6626 ,0|| 6626, 6627 , 6628
[0, 0, 1], -0.0, 6627 ,0|| 6627, 6628 , 6627

any_head_positioned True
15-14, Count=95, Err=0.0039


...

[1, 0, 0], -0.0, 6042 ,0|| 6042, 0 , 0
[1, 0, 0], -0.0, 6044 ,0|| 6044, 6042 , 0
[1, 0, 0], -0.0, 6045 ,0|| 6045, 6044 , 6042
[1, 0, 0], -0.0, 6048 ,0|| 6048, 6045 , 6044
[1, 0, 0], -0.1, 6049 ,0|| 6049, 6048 , 6045
[1, 0, 0], -0.1, 6048 ,0|| 6048, 6049 , 6048
[0, 1, 0], -0.0, 6017 ,0|| 6017, 0 , 0
[0, 1, 0], -0.0, 6014 ,0|| 6014, 6017 , 0
[0, 1, 0], -0.0, 6014 ,0|| 6014, 6014 , 6017
[0, 1, 0], -0.0, 6019 ,0|| 6014, 6017 , 6019
[0, 1, 0], -0.0, 6020 ,0|| 6017, 6019 , 6020
[0, 1, 0], -0.0, 6022 ,0|| 6019, 6020 , 6022
[0, 1, 0], 0.0, 6022 ,0|| 6020, 6022 , 6022
[0, 1, 0], 0.0, 6024 ,0|| 6022, 6022 , 6024
[0, 1, 0], 0.0, 6026 ,0|| 6022, 6024 , 6026
[0, 1, 0], 0.0, 6026 ,0|| 6024, 6026 , 6026
[0, 1, 0], 0.1, 6028 ,0|| 6026, 6026 , 6028
[0, 1, 0], 0.1, 6032 ,0|| 6026, 6028 , 6032
[0, 1, 0], 0.1, 6033 ,0|| 6028, 6032 , 6033
[0, 1, 0], 0.1, 6034 ,0|| 6032, 6033 , 6034
[0, 1, 0], 0.1, 6035 ,0|| 6033, 6034 , 6035
[0, 1, 0], 0.1, 6039 ,0|| 6034, 6035 , 6039
[0, 1, 0], 0.1, 6041 ,0|| 6035, 6039 ,

[1, 0, 0], -0.0, 5331 ,0|| 5331, 0 , 0
[1, 0, 0], -0.0, 5328 ,0|| 5328, 5331 , 0
[1, 0, 0], -0.0, 5328 ,0|| 5328, 5328 , 5331
[1, 0, 0], -0.0, 5328 ,0|| 5328, 5331 , 5328
[0, 1, 0], -0.0, 5362 ,0|| 5362, 0 , 0
[0, 1, 0], -0.0, 5361 ,0|| 5361, 5362 , 0
[0, 1, 0], -0.0, 5363 ,0|| 5363, 5361 , 5362
[0, 1, 0], -0.0, 5368 ,0|| 5368, 5363 , 5361
[0, 1, 0], -0.1, 5373 ,0|| 5373, 5368 , 5363
[0, 1, 0], -0.1, 5372 ,0|| 5372, 5373 , 5368
[0, 0, 1], -0.0, 5253 ,0|| 5253, 0 , 0
[0, 0, 1], -0.0, 5255 ,0|| 5255, 5253 , 0
[0, 0, 1], -0.0, 5256 ,0|| 5256, 5255 , 5253
[0, 0, 1], -0.0, 5257 ,0|| 5257, 5256 , 5255
[0, 0, 1], -0.1, 5254 ,0|| 5254, 5257 , 5256

any_head_positioned True
12-11, Count=46, Err=0.0039


[1, 0, 0], -0.0, 2905 ,0|| 2905, 0 , 0
[1, 0, 0], -0.0, 2904 ,0|| 2904, 2905 , 0
[1, 0, 0], -0.0, 2902 ,0|| 2902, 2904 , 2905
[1, 0, 0], -0.0, 2905 ,0|| 2904, 2905 , 2905
[1, 0, 0], -0.0, 2905 ,0|| 2905, 2905 , 2905
[1, 0, 0], -0.0, 2904 ,1|| 2904, 2905 , 2905
[1, 0, 0], -0.0, 2905 ,1|| 2905, 2905 , 2905
[0, 1, 0], -0.0, 2920 ,0|| 2920, 0 , 0
[0, 1, 0], -0.0, 2921 ,0|| 2921, 2920 , 0
[0, 1, 0], -0.0, 2919 ,0|| 2919, 2921 , 2920
[0, 0, 1], -0.0, 2923 ,0|| 2923, 0 , 0
[0, 0, 1], -0.0, 2920 ,0|| 2920, 2923 , 0
[0, 0, 1], -0.0, 2922 ,0|| 2922, 2920 , 2923
[0, 0, 1], -0.0, 2924 ,0|| 2924, 2922 , 2920
[0, 0, 1], -0.1, 2923 ,0|| 2923, 2924 , 2922

any_head_positioned True
11-10, Count=18, Err=0.0063


[1, 0, 0], -0.0, 2223 ,0|| 2223, 0 , 0
[1, 0, 0], -0.0, 2225 ,0|| 2225, 2223 , 0
[1, 0, 0], -0.0, 2227 ,0|| 2227, 2225 , 2223
[1, 0, 0], -0.0, 2224 ,0|| 2224, 2227 , 2225
[0, 1, 0], -0.0, 2210 ,0|| 2210, 0 , 0
[0, 1, 0], -0.0, 2212 ,0|| 2212, 2210 , 0
[0, 1, 0], -0.0, 2216 ,0|| 2216, 2212 , 2210
[0, 1, 0], -0.0, 2216 ,0|| 2216, 2216 , 2212
[0, 1, 0], -0.1, 2216 ,0|| 2216, 2216 , 2216
[0, 1, 0], -0.1, 2218 ,0|| 2218, 2216 , 2216
[0, 1, 0], -0.1, 2219 ,0|| 2219, 2218 , 2216
[0, 1, 0], -0.1, 2220 ,0|| 2220, 2219 , 2218
[0, 1, 0], -0.1, 2219 ,0|| 2219, 2220 , 2219
[0, 0, 1], -0.0, 2226 ,0|| 2226, 0 , 0
[0, 0, 1], -0.0, 2228 ,0|| 2228, 2226 , 0
[0, 0, 1], -0.0, 2232 ,0|| 2232, 2228 , 2226
[0, 0, 1], -0.0, 2230 ,0|| 2230, 2232 , 2228

any_head_positioned True
9-8, Count=10, Err=0.0248


[1, 0, 0], -0.0, 6531 ,0|| 6531, 0 , 0
[1, 0, 0], -0.0, 6532 ,0|| 6532, 6531 , 0
[1, 0, 0], -0.0, 6535 ,0|| 6535, 6532 , 6531
[1, 0, 0], -0.0, 6535 ,0|| 6535, 6535 , 6532
[1, 0, 0], -0.1, 6540 ,0|| 6540, 6535 , 6535
[1, 0, 0], -0.1, 6540 ,0|| 6540, 6540 , 6535
[1, 0, 0], -0.1, 6542 ,0|| 6542, 6540 , 6540
[1, 0, 0], -0.1, 6538 ,0|| 6538, 6542 , 6540
[0, 1, 0], -0.0, 6406 ,0|| 6406, 0 , 0
[0, 1, 0], -0.0, 6403 ,0|| 6403, 6406 , 0
[0, 1, 0], -0.0, 6398 ,0|| 6398, 6403 , 6406
[0, 1, 0], -0.0, 6406 ,0|| 6403, 6406 , 6406
[0, 1, 0], -0.0, 6407 ,0|| 6406, 6406 , 6407
[0, 1, 0], -0.0, 6409 ,0|| 6406, 6407 , 6409
[0, 1, 0], 0.0, 6411 ,0|| 6407, 6409 , 6411
[0, 1, 0], 0.0, 6411 ,0|| 6409, 6411 , 6411
[0, 1, 0], 0.0, 6414 ,0|| 6411, 6411 , 6414
[0, 1, 0], 0.0, 6418 ,0|| 6411, 6414 , 6418
[0, 1, 0], 0.1, 6419 ,0|| 6414, 6418 , 6419
[0, 1, 0], 0.1, 6419 ,0|| 6418, 6419 , 6419
[0, 1, 0], 0.1, 6418 ,0|| 6419, 6419 , 6418
[0, 1, 0], 0.1, 6418 ,1|| 6418, 6419 , 6419
[0, 1, 0], 0.1, 6418 ,1|| 6419, 6419

In [7]:
mhead.create_mesh('mesh')


completed


In [19]:
for i, link in enumerate(mhead.links):
    print (i)
    link.print_short()

0
2-1, Count=155, Err=0.0007
1
3-2, Count=111, Err=0.0018
2
4-3, Count=66, Err=0.0041
3
5-4, Count=48, Err=0.0054
4
6-5, Count=53, Err=0.0055
5
7-6, Count=55, Err=0.0024
6
8-7, Count=26, Err=0.0031
7
9-8, Count=10, Err=0.0248
8
10-9, Count=5, Err=nan
9
11-10, Count=18, Err=0.0063
10
12-11, Count=46, Err=0.0039
11
13-12, Count=56, Err=0.0038
12
14-13, Count=36, Err=0.0057
13
15-14, Count=95, Err=0.0039
14
1-15, Count=65, Err=0.0044


In [20]:
mhead.reset_all_head_positions()
mhead.transform_from_link(mhead.links[8])
mhead.create_spheres()

[1, 0, 0], -0.0, 5948 ,0|| 5948, 0 , 0
[1, 0, 0], -0.0, 5948 ,0|| 5948, 5948 , 0
[1, 0, 0], -0.0, 5948 ,0|| 5948, 5948 , 5948
[1, 0, 0], -0.0, 5948 ,0|| 5948, 5948 , 5948
[1, 0, 0], -0.1, 5949 ,0|| 5949, 5948 , 5948
[1, 0, 0], -0.1, 5953 ,0|| 5953, 5949 , 5948
[1, 0, 0], -0.1, 5955 ,0|| 5955, 5953 , 5949
[1, 0, 0], -0.1, 5956 ,0|| 5956, 5955 , 5953
[1, 0, 0], -0.1, 5956 ,0|| 5956, 5956 , 5955
[1, 0, 0], -0.1, 5959 ,0|| 5959, 5956 , 5956
[1, 0, 0], -0.1, 5960 ,0|| 5960, 5959 , 5956
[1, 0, 0], -0.1, 5959 ,0|| 5959, 5960 , 5959
[0, 1, 0], -0.0, 6052 ,0|| 6052, 0 , 0
[0, 1, 0], -0.0, 6052 ,0|| 6052, 6052 , 0
[0, 1, 0], -0.0, 6048 ,0|| 6048, 6052 , 6052
[0, 1, 0], -0.0, 6053 ,0|| 6052, 6052 , 6053
[0, 1, 0], -0.0, 6055 ,0|| 6052, 6053 , 6055
[0, 1, 0], -0.0, 6054 ,0|| 6053, 6055 , 6054
[0, 0, 1], -0.0, 5894 ,0|| 5894, 0 , 0
[0, 0, 1], -0.0, 5890 ,0|| 5890, 5894 , 0
[0, 0, 1], -0.0, 5890 ,0|| 5890, 5890 , 5894
[0, 0, 1], -0.0, 5898 ,0|| 5890, 5894 , 5898
[0, 0, 1], -0.0, 5898 ,0|| 5894, 5898

In [32]:
mhead.create_mesh("mesh_head")


completed


In [ ]:
# bad 11 12
# 12-11, Count=53, Err=0.0077

second

In [ ]:
#using edge diameter 20, 2 nearest neighbors, 0.9 threshold
#using edge diameter 20, 1 nearest neighbors, 0.4 threshold

In [31]:
mhead.reset_all_head_positions()
maxhead = 15
for idx in range(maxhead):
    mhead.transform_from_link(mhead.links[idx])
    mhead.links[idx].print_short()
center, distances=mhead.left_eye_deviation()
np.mean(distances)

mhead.create_spheres(0.5,"seq_2_mergeSet_{}".format(maxhead))

2-1, matches=967, Count=820, Err=0.0015
3-2, matches=812, Count=591, Err=0.0024
4-3, matches=702, Count=494, Err=0.0027
5-4, matches=652, Count=445, Err=0.0028
6-5, matches=606, Count=461, Err=0.0023
7-6, matches=308, Count=177, Err=0.0038
8-7, matches=168, Count=109, Err=0.0025
9-8, matches=134, Count=85, Err=0.0024
10-9, matches=161, Count=64, Err=0.0036
11-10, matches=165, Count=85, Err=0.0023
12-11, matches=85, Count=14, Err=0.0160
13-12, matches=211, Count=28, Err=0.0145
14-13, matches=741, Count=474, Err=0.0031
15-14, matches=1060, Count=927, Err=0.0012
1-15, matches=1017, Count=851, Err=0.0015
0 153974
coordinate of the left eye: [ 0.00277377 -0.09670738 -0.06905971] in frame 0
1 150143
coordinate of the left eye: [ 0.03848441 -0.14546269 -0.00836436] in frame 1
2 150124
coordinate of the left eye: [ 0.03808846 -0.14535251 -0.00839116] in frame 2
13 147576
coordinate of the left eye: [ 0.00752822 -0.09694682 -0.0707703 ] in frame 13
14 150775
coordinate of the left eye: [ 0.0041